### **1. 특징 추출 (Feature Extraction)**

-   **추출기 모델 정의**
    -   연구의 일관성 및 특징의 질을 확보하기 위해, 파트 1에서 가장 우수한 분류 성능을 보였던 **ResNet50 미세조정 모델**을 특징 추출기로 활용할 계획입니다.
    -   모델의 최종 분류층(`softmax`)을 제거하고, 그 직전의 `GlobalAveragePooling2D` 층의 출력을 각 이미지의 최종 특징 벡터로 정의합니다. 이 1024차원의 벡터는 이미지의 전반적인 스타일 정보를 압축한, 의미적으로 풍부한 표현(Semantically Rich Representation)으로 간주됩니다.

-   **특징 벡터 생성**
    -   전체 데이터셋(400장)의 모든 이미지를 위에서 정의한 특징 추출기 모델에 통과시켜, `[400, 1024]` 형태의 특징 벡터 행렬을 생성할 것입니다. 이 행렬이 후속 비지도 학습 분석의 기반 데이터가 됩니다.

### **2. 최적 군집 수(k) 결정: 엘보우 기법**

-   **분석의 목적**
    -   비지도 학습의 객관성을 확보하기 위해, 우리가 이미 알고 있는 클래스 수(4)를 선험적으로 사용하는 대신, 데이터의 분포로부터 최적의 군집 수를 도출하는 과정을 거칠 것입니다.
    -   이를 위해 K-Means 군집화에서 가장 널리 사용되는 휴리스틱 기법인 **엘보우 기법(Elbow Method)**을 적용하여, 데이터에 가장 적합한 `k`값을 결정합니다.

-   **분석 방법**
    -   `k`를 1부터 10까지 순차적으로 증가시키며 K-Means 알고리즘을 반복 실행합니다.
    -   각 `k`값에 대해 군집 내 오차 제곱합(WCSS, Within-Cluster Sum of Squares)을 계산합니다. WCSS는 군집 내 데이터 포인트들이 얼마나 조밀하게 모여있는지를 나타내는 척도입니다.
    -   `k`값의 변화에 따른 WCSS 감소량을 그래프로 시각화하여, 감소율이 급격히 완만해지는 변곡점, 즉 '엘보우(Elbow)' 지점을 찾습니다. 이 지점이 군집의 수를 늘렸을 때 얻는 이점이 줄어드는 최적의 `k`값으로 해석됩니다.
    -   `[Figure 5]`에서는 이 엘보우 곡선 그래프를 제시하여 최적의 `k`값을 선택하는 과정을 논리적으로 증명할 예정입니다.


### **3. K-Means 군집화 및 시각적 분석**

-   **군집화 수행**
    -   엘보우 기법을 통해 도출된 최적의 `k`값을 사용하여, 1024차원의 특징 벡터 전체에 대해 K-Means 군집화를 최종적으로 수행합니다.

-   **t-SNE를 이용한 시각화**
    -   모델이 생성한 고차원의 군집 결과를 사람이 직관적으로 이해할 수 있도록, 대표적인 비선형 차원 축소 기법인 **t-SNE**를 적용하여 특징 벡터를 2차원 공간에 시각화할 계획입니다.
    -   t-SNE는 고차원 공간에서 가까웠던 데이터 포인트들이 2차원 공간에서도 가깝게 유지되도록 하여, 데이터의 군집 구조를 시각적으로 표현하는 데 매우 효과적입니다.
    -   `[Figure 6]`에서는 K-Means가 할당한 군집 레이블을 각기 다른 색상으로 t-SNE 공간상에 표시할 것입니다. 이를 통해 각 군집이 얼마나 명확하게 분리되는지, 군집 내 응집도는 어떠한지 시각적으로 확인할 수 있습니다.

-   **정성적 결과 해석**
    -   각 군집이 어떤 시각적, 스타일적 특성을 공유하는지 구체적으로 파악하기 위해, 군집별로 5~8개의 이미지를 무작위로 샘플링하여 `[Figure 7]`에 제시할 것입니다.
    -   예를 들어, **Cluster 0**에서는 무채색과 단순한 실루엣의 '미니멀' 스타일 이미지가 다수 발견되고, **Cluster 1**에서는 오버사이즈 핏과 화려한 그래픽의 '스트릿' 스타일 이미지가 주로 나타나는 등의 경향을 분석합니다.
    -   이러한 정성적 분석을 통해, 레이블 없이 데이터의 특징만으로도 의미 있는 스타일 그룹을 성공적으로 발견했음을 입증하고, 각 군집의 특성을 상세히 기술할 예정입니다.

